In [3]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time

class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.tick_data = []  # This will hold all collected tick data
        self.new_ticks = []  # This will hold the current batch of tick data
        self.collecting = False

    def historicalTicks(self, reqId, ticks, done):
        print("Historical Tick Data. ReqId:", reqId)
        self.new_ticks = []  # Clear the new_ticks list for the current batch
        for tick in ticks:
            print(f"Tick: {tick.time}, Price: {tick.price}, Size: {tick.size}")
            self.new_ticks.append((tick.time, tick.price, tick.size))
        print("Done with historical ticks")
        self.collecting = False

    def historicalTicksLast(self, reqId, ticks, done):
        print("Historical Tick Last Data. ReqId:", reqId)
        self.new_ticks = []  # Clear the new_ticks list for the current batch
        for tick in ticks:
            print(f"Tick: {tick.time}, Price: {tick.price}, Size: {tick.size}")
            self.new_ticks.append((tick.time, tick.price, tick.size))
        print("Done with historical ticks last")
        if done:
            self.collecting = False

def run_loop():
    app.run()

app = IBApi()
app.connect("127.0.0.1", 7497, 0)

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1)  # Sleep to ensure connection is established

def format_time(t):
    if type(t) == str: return t
    format = "%Y%m%d %H:%M:%S"
    return time.strftime(format, time.localtime(t))

def now():
    return format_time(time.time())

contract = Contract()
contract.symbol = "ES"
contract.secType = "CONTFUT"
contract.exchange = "CME"
contract.currency = "USD"
contract.lastTradeDateOrContractMonth = "202409"  # Example for September 2024 contract

def request_historical_ticks(start_time):
    """Request historical tick data starting from start_time."""
    app.collecting = True
    app.reqHistoricalTicks(
        reqId=1,
        contract=contract,
        startDateTime=start_time,
        endDateTime="",
        numberOfTicks=1000,
        whatToShow="TRADES",
        useRth=0,
        ignoreSize=False,
        miscOptions=[]
    )

def collect_ticks():
    """Collect tick data in batches without missing any data."""
    start_time = "20240724 06:20:04"
    previous_overlap_count = 0
    
    app.collecting = True
    app.reqHistoricalTicks(
        reqId=1,
        contract=contract,
        startDateTime="",
        endDateTime=now(),
        numberOfTicks=1000,
        whatToShow="TRADES",
        useRth=0,
        ignoreSize=False,
        miscOptions=[]
    )
    while app.collecting:
        print("Collecting initial batch...")
        time.sleep(1)


    while True:
        request_historical_ticks(start_time)
        while app.collecting:
            time.sleep(1)
        print(f"Collected {len(app.new_ticks)} ticks")
        
        if app.tick_data:
            # Determine the second-to-last timestamp from the overall tick_data
            second_to_last_time = app.tick_data[-2][0] if len(app.tick_data) > 1 else app.tick_data[-1][0]
        else:
            second_to_last_time = start_time

        # Calculate the number of overlapping timestamps in the new batch
        last_time = int(app.tick_data[-1][0]) if app.tick_data else None
        overlap_count = sum(1 for d in app.new_ticks if d[0] == last_time)
        
        # Filter overlapping data from new_ticks
        filtered_new_ticks = [d for d in app.new_ticks if d[0] != last_time]
        
        # Append filtered new ticks to the main tick data
        app.tick_data.extend(filtered_new_ticks)

        # Update the start_time for the next batch
        print(second_to_last_time, type(second_to_last_time))
        start_time = format_time(second_to_last_time)
        # start_time = second_to_last_time

        # Measure the difference in the number of overlapping timestamps
        overlap_difference = overlap_count - previous_overlap_count
        previous_overlap_count = overlap_count

        print(f"Overlap Difference: {overlap_difference}")

        #append to data.csv
        with open('data.csv', 'a') as f:
            for tick in app.new_ticks:
                f.write(f"{tick[0]},{tick[1]},{int(tick[2])}\n")
        
        time.sleep(10)  # Adjust based on rate limit

# Start the cyclic data collection
# collect_ticks_thread = threading.Thread(target=collect_ticks, daemon=True)
# collect_ticks_thread.start()
collect_ticks()
# This will run indefinitely, collecting tick data

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj
ERROR 1 2174 Warning: You submitted request with date-time attributes without explicit time zone. Please switch to use yyyymmdd-hh:mm:ss in UTC or use instrument time zone, like US/Eastern. Implied time zone functionality will be removed in the next API release


Historical Tick Last Data. ReqId: 1
Tick: 1724178995, Price: 5628.5, Size: 1
Tick: 1724178995, Price: 5628.25, Size: 1
Tick: 1724178995, Price: 5628.5, Size: 2
Tick: 1724178995, Price: 5628.25, Size: 13
Tick: 1724178997, Price: 5628.5, Size: 3
Tick: 1724178998, Price: 5628.25, Size: 1
Tick: 1724178999, Price: 5628.25, Size: 12
Tick: 1724178999, Price: 5628.25, Size: 1
Tick: 1724178999, Price: 5628.25, Size: 1
Tick: 1724178999, Price: 5628.25, Size: 1
Tick: 1724178999, Price: 5628.25, Size: 5
Tick: 1724178999, Price: 5628.25, Size: 1
Tick: 1724178999, Price: 5628.25, Size: 1
Tick: 1724178999, Price: 5628.25, Size: 1
Tick: 1724178999, Price: 5628.25, Size: 1
Tick: 1724179000, Price: 5628.25, Size: 1
Tick: 1724179000, Price: 5628.25, Size: 2
Tick: 1724179001, Price: 5628.5, Size: 1
Tick: 1724179001, Price: 5628.5, Size: 10
Tick: 1724179001, Price: 5628.5, Size: 4
Tick: 1724179001, Price: 5628.5, Size: 5
Tick: 1724179001, Price: 5628.5, Size: 2
Tick: 1724179001, Price: 5628.5, Size: 1
Tick

KeyboardInterrupt: 

ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj
ERROR -1 2103 Market data farm connection is broken:usfarm.nj
ERROR -1 2105 HMDS data farm connection is broken:ushmds
ERROR -1 2103 Market data farm connection is broken:cashfarm
ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2103 Market data farm connection is broken:usfuture
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1

In [ ]:
app.tick_data

[]

In [2]:
app.disconnect()

In [ ]:
now()

'20240724 06:20:04'